In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
import plotly.express as px
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, ConfusionMatrixDisplay, silhouette_score

# Part 1: Forest Cover Type
## Section A:Data Exploration & Visualization
### Data Loading and Initial Exploration

In [ ]:
# Load the dataset
df0 = pd.read_csv("treetypes.csv", index_col=0)
df = df0.reset_index()
# Display the first few rows of the dataset
print("First 5 rows of the dataset:")
df.head()

In [ ]:
df.info()

In [ ]:
# Generate descriptive statistics
print("Descriptive Statistics:")
df.describe()

In [ ]:
# Check for missing values
print("Missing Values Count:")
df.isnull().sum()

This dataset contains 45,000 entries and 54 columns, with features like Aspect, Slope, Hillshade at different times of day, and distances to hydrology, roadways, and fire points. It also includes many one-hot encoded soil and wilderness types. Most features are numeric, while categorical features are represented as binary columns. Elevation was originally used as the index, but we modified the dataset to treat it as a feature, resetting the index to default (0, 1, 2, ...). The dataset appears clean and ready for modeling. We will proceed with any required preprocessing in section B.

### Understanding the Dataset

In [ ]:
# Calculate total counts of each Wilderness_Area (1–4) per tree type
wilderness_cols = [col for col in df.columns if col.startswith("Wilderness_Area")]
wilderness_counts = df.groupby("label")[wilderness_cols].sum()

# Rename columns for clearer labels
wilderness_counts.columns = [col.replace("Wilderness_Area", "Area ") for col in wilderness_counts.columns]

# Plot: stacked bar chart
plt.figure(figsize=(10, 6))
wilderness_counts.plot(kind="bar", stacked=True, colormap="Set2", figsize=(10, 6))
plt.title("Distribution of Wilderness Areas per Tree Type")
plt.xlabel("Tree Type (Label)")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.legend(title="Wilderness Area", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

##### Observation:
This stacked bar plot shows the distribution of wilderness areas across tree types. Label 3 is heavily concentrated in Wilderness Area 4, making this feature a strong predictor for identifying Label 3. In contrast, Labels 1 and 2 appear in similar proportions across Areas 1–3, offering little discriminative value. This confirms that Wilderness Area 4 should be treated as a signal feature, while the others may be less useful.

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x="Slope", y="Elevation", hue="label", palette="tab10", alpha=0.5)
plt.title("Elevation vs. Slope by Tree Type")
plt.xlabel("Slope")
plt.ylabel("Elevation")
plt.legend(title="Tree Type", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

##### Observation:
This scatter plot visualizes the relationship between elevation and slope across tree types. Label 3 stands out clearly in the low-elevation, low-slope region, while Labels 1 and 2 overlap heavily. This confirms elevation’s strength as a predictive feature and suggests that slope alone is not very informative — most of the observed separation is driven by elevation. Therefore, while slope may contribute minor contextual value in multivariate models, it is not a strong feature by itself.

In [ ]:
# Prepare the soil usage matrix
soil_cols = [col for col in df.columns if col.startswith("Soil_Type")]
soil_usage = df.groupby("label")[soil_cols].sum()

# Melt the dataframe to long format
soil_usage_long = soil_usage.reset_index().melt(id_vars='label', var_name='Soil_Type', value_name='Count')

# Get top 5 soil types per tree type — updated to avoid the warning
top5_soils_per_label = (
    soil_usage_long
    .sort_values(["label", "Count"], ascending=[True, False])
    .groupby("label", group_keys=False)
    .head(5)
)

# Plot
plt.figure(figsize=(12, 6))
sns.barplot(data=top5_soils_per_label, x="label", y="Count", hue="Soil_Type", palette="tab20")
plt.title("Top 5 Soil Types per Tree Type")
plt.xlabel("Tree Type (Label)")
plt.ylabel("Occurrences")
plt.legend(title="Soil Type", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

##### Observation:
This plot shows the top 5 most common soil types per tree label. It reveals that Label 3 strongly favors certain soil types that are rare in other labels, making soil type a highly predictive feature for separating Label 3. Labels 1 and 2 share more overlap in soil usage but still show minor differences (e.g., Soil_Type22 for Label 1).

In [ ]:
df["Elevation"] = pd.to_numeric(df["Elevation"], errors="coerce")
df_1 = df[df["label"].isin([1, 2, 3])]  # Filter only labels 1, 2, 3
df_1 = df_1.dropna(subset=["Elevation"])  # Remove any null elevation

# Map labels to colors
label_color_map = {1: "blue", 2: "green", 3: "red"}
label_name_map = {1: "Label 1", 2: "Label 2", 3: "Label 3"}

# Plot KDEs manually by label
plt.figure(figsize=(8, 6))
for label in [1, 2, 3]:
    subset = df_1[df_1["label"] == label]
    sns.kdeplot(
        data=subset,
        x="Elevation",
        fill=True,
        alpha=0.4,
        linewidth=1.5,
        color=label_color_map[label],
        label=label_name_map[label]
    )

# Final plot settings
plt.title("Elevation Distribution by Tree Type (Labels 1, 2, 3)")
plt.xlabel("Elevation")
plt.ylabel("Density")
plt.legend(title="Tree Type")
plt.tight_layout()
plt.show()

##### Observation:
This KDE plot shows the elevation distributions for Labels 1, 2, and 3. Label 3 stands out with a clearly lower elevation range, making Elevation an excellent feature for isolating it. While Labels 1 and 2 overlap more heavily, Label 2 tends to appear at slightly higher elevations. This makes Elevation a strong but imperfect predictor across all labels, particularly useful for distinguishing Label 3. This insight supports using Elevation prominently in modeling and feature selection.

In [ ]:
selected_features = ['Horizontal_Distance_To_Hydrology', 'Horizontal_Distance_To_Fire_Points','Hillshade_Noon', 'Hillshade_9am', 'Hillshade_3pm', 'label']
df_subset = df[selected_features].copy()

# Filter only labels 1 and 2
df_subset = df_subset[df_subset["label"].isin([1, 2])]
df_subset["label"] = df_subset["label"].astype(str)  # ensure string for coloring

# Plot
sns.pairplot(df_subset, hue='label',
             palette={'1': 'lightgreen', '2': 'salmon'},
             plot_kws={'alpha': 1, 's': 20})

plt.suptitle('Pairplot of Selected Features by Tree Type (Label 1 vs 2)', y=1.02)
plt.tight_layout()
plt.show()

##### Observation:
This pairplot explores terrain and light-related features for Label 1 vs 2. All four features show substantial overlap between the labels, both individually and in combination. While this plot does not reveal strong separation, it’s important as it confirms that these features alone are not sufficient for distinguishing the two labels. This insight will guide our model design to rely on multi-feature combinations rather than isolated features.

## Section B - Data Pre-processing
p.s we do not need to clean the dataset before feature engineering since we have no null values and all data have the correct type
### feature engineering

In [ ]:
# Reload the dataset
df0 = pd.read_csv("treetypes.csv", index_col=0).reset_index()

In [ ]:
# 1. Hydrology Vertical/Horizontal Ratio
df0["Hydrology_VH_Ratio"] = df0["Vertical_Distance_To_Hydrology"] / (df0["Horizontal_Distance_To_Hydrology"] + 1)

# 2. Total Horizontal Distance (Fire + Road + Water)
df0["Total_Horiz_Distance"] = (
    df0["Horizontal_Distance_To_Hydrology"] +
    df0["Horizontal_Distance_To_Fire_Points"] +
    df0["Horizontal_Distance_To_Roadways"]
)

# 3. Hillshade Range (max - min of 3 times)
hillshade_cols = ["Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm"]
df0["Hillshade_Range"] = df0[hillshade_cols].max(axis=1) - df0[hillshade_cols].min(axis=1)

# 4. Downhill to Water (binary: 1 if slope down to hydrology)
df0["Downhill_To_Water"] = (df0["Vertical_Distance_To_Hydrology"] < 0).astype(float)

In [ ]:
# Return updated dataframe shape and columns added
df0.shape, df0.columns[0:].tolist()

| Feature Name                  | Formula / Description                                                                 | Reason to Add                                                                 |
|------------------------------|----------------------------------------------------------------------------------------|--------------------------------------------------------------------------------|
| `Total_Horiz_Distance`       | Sum of horizontal distances to Hydrology, Fire Points, and Roadways                   | Captures total remoteness from infrastructure — affects environment and tree growth |
| `Hydrology_VH_Ratio`         | Vertical ÷ (Horizontal + 1) distance to hydrology                                     | Approximates terrain slope toward water — steeper slopes may affect soil or runoff  |
| `Downhill_To_Water`          | Boolean: 1 if Vertical Distance to Hydrology < 0                                      | Indicates whether terrain descends toward water — may influence moisture or erosion |
| `Hillshade_Range`            | Max - Min of Hillshade at 9am, Noon, 3pm                                              | Captures variation in sunlight exposure across the day — relevant to shade tolerance |

### Data Cleaning after geature engineering:

In [ ]:
df0.info()

In [ ]:
# Copy the relevant features to scale
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df0[["Total_Horiz_Distance", "Hydrology_VH_Ratio"]])
scaled_df = pd.DataFrame(scaled_features, columns=["Total_Horiz_Distance_Scaled", "Hydrology_VH_Ratio_Scaled"])

# Merge the scaled features into the original DataFrame
df0 = pd.concat([df0, scaled_df], axis=1)

In [ ]:
# Check for any columns that have only a single unique value
useless_cols = [col for col in df0.columns if df0[col].nunique() <= 1]
print(useless_cols)

In [ ]:
df0.drop(columns=["Soil_Type15", "Soil_Type37"], inplace=True)
df0.drop(columns=["Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm"], inplace=True)
df0.drop(columns=["Horizontal_Distance_To_Roadways","Horizontal_Distance_To_Fire_Points"], inplace=True)

In [ ]:
print('Data Cleaning Completed\n')
df0.info()

In [ ]:
df0.describe().T.sort_values('std', ascending=False)

### Full Data Cleaning and Preprocessing Summary

This section outlines the complete data preparation process used to convert the raw `treetypes.csv` dataset into a model-ready format for tree classification. All decisions were made to ensure compatibility with models such as KNN, Random Forest, Gradient Boosting, and PCA, while preserving informative environmental and terrain features.

#### 1. Feature Engineering

We constructed several informative features based on domain intuition and terrain analysis:

- `Total_Horiz_Distance`: Sum of horizontal distances to hydrology, fire points, and roadways. This measures a tree's overall remoteness from manmade and natural infrastructure.
- `Hydrology_VH_Ratio`: Vertical ÷ (Horizontal + 1) distance to hydrology. This ratio approximates the steepness of terrain sloping into water — important for soil runoff and water exposure.
- `Downhill_To_Water`: Boolean flag equal to 1 if a tree is located downhill from hydrology (i.e., `Vertical_Distance_To_Hydrology` < 0). This may affect water saturation and erosion.
- `Hillshade_Range`: The difference between the maximum and minimum hillshade across 9am, Noon, and 3pm. Captures terrain-related variation in sunlight exposure.

All engineered features are numeric and additive — enhancing model input while preserving interpretability.

#### 2. Scaling for PCA and KNN

Two continuous features were scaled using `StandardScaler` to ensure compatibility with distance-based models:

- `Total_Horiz_Distance_Scaled`: Standardized version of total horizontal distance.
- `Hydrology_VH_Ratio_Scaled`: Standardized version of the hydrology slope ratio.

The original versions were retained for tree-based models which do not require scaling.

#### 3. Redundant Feature Removal

Two soil type features were found to be entirely unused in the dataset (i.e., all values were zero):

- `Soil_Type15`
- `Soil_Type37`

Features that were non-informative, textual, or already incorporated through feature engineering:

- `Hillshade_9am`
- `Hillshade_Noon`
- `Hillshade_3pm`
- `Horizontal_Distance_To_Roadways`
- `Horizontal_Distance_To_Fire_Points`

These were dropped to reduce feature noise and improve efficiency.

#### 4. Final Dataset Verification

- All features are numeric (`float64` or `int64`)
- No missing values, infinite values, or constant columns remain
- Dataset retains full structure and balance: ~45,000 samples × 50+ informative features
- All engineered and scaled features are now available for modeling in Sections C and D

The resulting dataset is now fully cleaned, transformed, and ready for model training, PCA, and classification evaluation.

## Section C - Classification and Clustering

for classification We have chosen these three models:
- random forest
- GBoost with Tree
- Neural Networks 

for clustering we have chosen these two models:
- K-Means
- Agglomerative Clustering

## Section C.1 - Classification

### Section C.1.1 - Setup and Data Preparation

In this section, we prepare the dataset for modeling. We define the features (X) and target (y),
split the data into train, validation, and test sets using an 80/10/10 split, as required by the assignment.

In [ ]:
# Features and target
X = df0.drop(columns=['label'])
y = df0['label']

# Split: 80/10/10
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

### Section C.1.2 - Model: Random Forest

We train a Random Forest classifier using GridSearchCV to tune `n_estimators`, `max_depth`, and apply `class_weight='balanced'`
for handling class imbalance. Evaluation is based on Macro F1 to ensure fairness to both classes.


In [ ]:
rf_params = {'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'class_weight': ['balanced']}

rf_gs = GridSearchCV(RandomForestClassifier(random_state=42),rf_params,scoring='f1_macro', cv=3,n_jobs=-1)
rf_gs.fit(X_train, y_train)
print("Best RF Params:", rf_gs.best_params_)

In [ ]:
rf_best = rf_gs.best_estimator_
y_val_pred_rf = rf_best.predict(X_val)
y_test_pred_rf = rf_best.predict(X_test)

In [ ]:
ConfusionMatrixDisplay.from_estimator(rf_best, X_val, y_val, cmap='Blues')
plt.title("Random Forest - Validation Confusion Matrix")
plt.show()

Random Forest Summary:
- Tuned using GridSearchCV with 3-fold CV.
- Best parameters were `class_weight='balanced'`, `max_depth=None`, and `n_estimators=200`.
- Performed strongly overall, with very high accuracy across all three classes.
- Class 3 had nearly perfect classification (1,495 correct predictions, only 5 misclassified as class 2).
- Most errors occurred between classes 1 and 2 (182 class 1 instances predicted as class 2, and 186 class 2 instances predicted as class 1), which is what was predicted since we've seen in section A how much they overlap.
- No confusion was observed between classes 1 and 3, indicating they are well-separated in feature space.

### Section C.1.3 - Model: Gradient Boosting

Next, we train a Gradient Boosting classifier, tuning tree depth, learning rate, and number of trees.
This model handles tends to perform well when tuned properly.

In [ ]:
# Step 1: Define parameter grid
gboost_params = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5]
}

# Step 2: GridSearchCV using macro F1 
gboost_gs = GridSearchCV(
    GradientBoostingClassifier(random_state=42),
    gboost_params,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1
)
gboost_gs.fit(X_train, y_train)

# Step 3: Get best hyperparameters from grid search
gboost_best = gboost_gs.best_estimator_

# Step 4: Make predictions
y_val_pred_gb = gboost_best.predict(X_val)
y_test_pred_gb = gboost_best.predict(X_test)

# Step 5: (Optional) Evaluate F1 scores
val_f1_gb = f1_score(y_val, y_val_pred_gb, average='macro')
test_f1_gb = f1_score(y_test, y_test_pred_gb, average='macro')

print("Validation Macro F1 Score:", val_f1_gb)
print("Test Macro F1 Score:", test_f1_gb)

In [ ]:
# Confusion Matrix - Gradient Boosting
ConfusionMatrixDisplay.from_estimator(gboost_best, X_val, y_val, cmap='Purples')
plt.title("GBoost - Validation Confusion Matrix")
plt.show()

Gradient Boosting Summary:
- Tuned tree depth, number of trees, and learning rate.
- ....
### Section C.1.4 - Model: Neural Networks

Neural Network rquire....

In [ ]:
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [ ]:
mlp_params = { 'hidden_layer_sizes': [(50,), (100,), (100, 50)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive']}

# Step 2: GridSearchCV using macro F1 to tune 
mlp_gs = GridSearchCV(MLPClassifier(max_iter=500, early_stopping=True, random_state=42) ,mlp_params,scoring='f1_macro', cv=3,n_jobs=-1)
mlp_gs.fit(X_train_scaled, y_train)
print("Best MLP Params:", mlp_gs.best_params_)

# Step 3: Get best hyperparameters from grid search
mlp_best = mlp_gs.best_estimator_

# Step 4: Make predictions
y_val_pred_mlp = mlp_best.predict(X_val_scaled)
y_test_pred_mlp = mlp_best.predict(X_test_scaled)

# Step 5: (Optional) Evaluate F1 scores
val_f1_mpl = f1_score(y_val, y_val_pred_mlp, average='macro')
test_f1_mpl = f1_score(y_test, y_test_pred_mlp, average='macro')

print("Validation Macro F1 Score:", val_f1_mpl)
print("Test Macro F1 Score:", test_f1_mpl)

In [ ]:
ConfusionMatrixDisplay.from_estimator(mlp_best, X_val, y_val, cmap='Blues')
plt.title("Neural Networks (Multi Layer Perception Classifier) - Validation Confusion Matrix")
plt.show()

Neural Networks summary:
- .....
- .....

### Section C.1.5 - VotingClassifier Ensemble 

We ensemble the three models using a VotingClassifier.
We refit the SVM using `probability=True`, which is required for soft voting.
Although we use hard voting here, this configuration gives us flexibility to easily switch.

In [ ]:
voting = VotingClassifier(estimators=[('rf', rf_best), ('gb', gboost_best), ('mlp', mlp_best)], voting='soft')

voting.fit(X_train_scaled, y_train)

y_val_pred_vote = voting.predict(X_val_scaled)
print("VotingClassifier - Validation F1:", f1_score(y_val, y_val_pred_vote, average='macro'), "\n")
print(classification_report(y_val, y_val_pred_vote))

# Test predictions
y_test_pred_vote = voting.predict(X_test_scaled)
print("VotingClassifier - Test F1:", f1_score(y_test, y_test_pred_vote, average='macro'), "\n")
print(classification_report(y_test, y_test_pred_vote))


Voting ensemble summary:

- ...
- ...
- ...

### Section C.1.6 - Model Comparison and Evaluation

We compare all three models (RF, GBoost, Nueral Networks (MLP)) using both accuracy and macro F1.
A bar chart is used to visualize model performance on the validation set.

In [ ]:
# Generate macro F1
metrics_summary = {
    'Model': ['Random Forest', 'GBoost', 'Neural Networks'],
    'Val Accuracy': [
        accuracy_score(y_val, y_val_pred_rf),
        accuracy_score(y_val, y_val_pred_gb),
        accuracy_score(y_val, y_val_pred_mlp)
    ],
    'Val Macro F1': [
        f1_score(y_val, y_val_pred_rf, average='macro'),
        f1_score(y_val, y_val_pred_gb, average='macro'),
        f1_score(y_val, y_val_pred_mlp, average='macro')
    ],
    'Test Accuracy': [
        accuracy_score(y_test, y_test_pred_rf),
        accuracy_score(y_test, y_test_pred_gb),
        accuracy_score(y_test, y_test_pred_mlp)
    ],
    'Test Macro F1': [
        f1_score(y_test, y_test_pred_rf, average='macro'),
        f1_score(y_test, y_test_pred_gb, average='macro'),
        f1_score(y_test, y_test_pred_mlp, average='macro')
    ]
}
# Add VotingClassifier to the summary
metrics_summary['Model'].append('Voting Ensemble')
metrics_summary['Val Accuracy'].append(accuracy_score(y_val, y_val_pred_vote))
metrics_summary['Val Macro F1'].append(f1_score(y_val, y_val_pred_vote, average='macro'))
metrics_summary['Test Accuracy'].append(accuracy_score(y_test, y_test_pred_vote))
metrics_summary['Test Macro F1'].append(f1_score(y_test, y_test_pred_vote, average='macro'))

summary_df = pd.DataFrame(metrics_summary)

In [ ]:
# Validation Plot
summary_df.set_index('Model')[['Val Accuracy', 'Val Macro F1']].plot(kind='bar', figsize=(8, 5), color=['steelblue', 'seagreen'])
plt.title('Model Comparison: Accuracy and Macro F1')
plt.ylabel('Score')
plt.ylim(0.75, 0.90)
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Test Plot
summary_df.set_index('Model')[['Test Accuracy', 'Test Macro F1']].plot(kind='bar', figsize=(8, 5), color=['orange', 'tomato'])
plt.title('Model Comparison: Test Accuracy and Macro F1')
plt.ylabel('Score')
plt.ylim(0.75, 0.90)
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

### Final Evaluation Summary


## Section C.2 - Clustering
